In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import time
from PIL import Image, ImageDraw

In [2]:
path = "./posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"
interpreter = tf.lite.Interpreter(path)
input_details = interpreter.get_input_details()
print(str(input_details))
output_details = interpreter.get_output_details()
print(str(output_details))

[{'name': 'sub_2', 'index': 93, 'shape': array([  1, 257, 257,   3], dtype=int32), 'shape_signature': array([  1, 257, 257,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'MobilenetV1/heatmap_2/BiasAdd', 'index': 87, 'shape': array([ 1,  9,  9, 17], dtype=int32), 'shape_signature': array([ 1,  9,  9, 17], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'MobilenetV1/offset_2/BiasAdd', 'index': 90, 'shape': array([ 1,  9,  9, 34], dtype=int32), 'shape_signature': array([ 1,  9,  9, 34], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': arra

In [5]:
for v in output_details:
    print(v)

{'name': 'MobilenetV1/heatmap_2/BiasAdd', 'index': 87, 'shape': array([ 1,  9,  9, 17], dtype=int32), 'shape_signature': array([ 1,  9,  9, 17], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'MobilenetV1/offset_2/BiasAdd', 'index': 90, 'shape': array([ 1,  9,  9, 34], dtype=int32), 'shape_signature': array([ 1,  9,  9, 34], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'MobilenetV1/displacement_fwd_2/BiasAdd', 'index': 84, 'shape': array([ 1,  9,  9, 32], dtype=int32), 'shape_signature': array([ 1,  9,  9, 32], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_paramet

In [6]:
# check outputstrids:  resolution = ((InputImageSize - 1) / OutputStride) + 1

output_stridex = (input_details[0]['shape'][1]-1)/(output_details[0]['shape'][1]-1)
output_stridey = (input_details[0]['shape'][2]-1)/(output_details[0]['shape'][2]-1)
print("outputstrides:", output_stridex, output_stridey)

outputstrides: 32.0 32.0


In [4]:
output_stride = 32
num_keypoints = 17

In [5]:
interpreter.allocate_tensors()

In [6]:
t1 = "/home/cp/Desktop/test1.png"
timg = Image.open(t1)
timg2 = timg.resize((257,257))
tim = np.array(timg2)
tim2 = np.expand_dims(tim, axis=0)

In [7]:
t1 = time.time()
interpreter.set_tensor(input_details[0]['index'], tim2.astype(np.float32))
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
t2 = time.time()
print("time:",t2-t1)

time: 0.11983823776245117


In [34]:
heatmaps = np.squeeze(interpreter.get_tensor(output_details[0]['index']))
offsets = np.squeeze(interpreter.get_tensor(output_details[1]['index']))

In [35]:
heatmaps.shape, offsets.shape

((23, 17, 17), (23, 17, 34))

In [60]:
def get_keypoints(heatmaps, offsets, output_stride=16.0):
#     scores = np.sigmoid(heatmaps)
    scores = heatmaps
    num_keypoints = scores.shape[2]
    heatmap_positions = []
    offset_vectors = []
    confidences = []
    for ki in range(0, num_keypoints ):
        x,y = np.unravel_index(np.argmax(scores[:,:,ki]), scores[:,:,ki].shape)
        confidences.append(scores[x,y,ki])
        offset_vector = (offsets[x,y,ki], offsets[x,y,num_keypoints+ki])
        heatmap_positions.append((x,y))
        offset_vectors.append(offset_vector)
    image_positions = np.add(np.array(heatmap_positions) * output_stride, offset_vectors)
    keypoints = [(i, pos, confidences[i]) for i, pos in enumerate(image_positions)]
    return keypoints

In [61]:
get_keypoints(heatmaps, offsets)

[(0, array([325.4461565 , 250.78218555]), -6.1496406),
 (1, array([354.15077996, 222.89347756]), -6.304964),
 (2, array([321.87555075, 249.6857872 ]), -5.4582505),
 (3, array([305.31788659,  -2.75290179]), -6.501217),
 (4, array([325.01177835, 246.78412104]), -5.8539906),
 (5, array([359.13035011,  99.73285699]), -5.4797087),
 (6, array([360.20594025,  62.11213291]), -5.6351075),
 (7, array([ 37.88518763, 210.92842484]), -6.895049),
 (8, array([331.95654869, 242.40388417]), -6.560242),
 (9, array([307.25256658,  -7.66912794]), -6.995436),
 (10, array([294.67760611,  -7.45043898]), -6.7330246),
 (11, array([354.75411129,  33.2149421 ]), -6.204076),
 (12, array([358.17596674,  49.67506397]), -5.9366326),
 (13, array([324.2558732 , 260.58329153]), -7.461457),
 (14, array([353.31652951, 258.56145549]), -7.1741033),
 (15, array([352.59026635,  92.53342628]), -7.398567),
 (16, array([295.53600168,  -7.39525414]), -7.115737)]

In [49]:
for i in range(num_keypoints):
    x,y = np.unravel_index(np.argmax(heatmaps[:,:,i]), heatmaps[:,:,i].shape) 
    conf = heatmaps[x,y,i]
    offset_vector = (offsets[x,y,i], offsets[x,y,num_keypoints+i])
    print(x,y,conf, offset_vector)

    pos_i = np.add(np.array([x,y]) * output_stride, offset_vector)
    print(pos_i)

20 15 -6.1496406 (5.4461565, 10.782186)
[325.4461565  250.78218555]
22 14 -6.304964 (2.15078, -1.1065224)
[354.15077996 222.89347756]
20 15 -5.4582505 (1.8755507, 9.685787)
[321.87555075 249.6857872 ]
19 0 -6.501217 (1.3178866, -2.7529018)
[305.31788659  -2.75290179]
20 15 -5.8539906 (5.0117784, 6.784121)
[325.01177835 246.78412104]
22 6 -5.4797087 (7.13035, 3.732857)
[359.13035011  99.73285699]
22 4 -5.6351075 (8.20594, -1.8878671)
[360.20594025  62.11213291]
2 12 -6.895049 (5.8851876, 18.928425)
[ 37.88518763 210.92842484]
20 15 -6.560242 (11.956549, 2.4038842)
[331.95654869 242.40388417]
19 0 -6.995436 (3.2525666, -7.669128)
[307.25256658  -7.66912794]
18 0 -6.7330246 (6.677606, -7.450439)
[294.67760611  -7.45043898]
22 2 -6.204076 (2.7541113, 1.2149421)
[354.75411129  33.2149421 ]
22 3 -5.9366326 (6.1759667, 1.675064)
[358.17596674  49.67506397]
20 16 -7.461457 (4.255873, 4.5832915)
[324.2558732  260.58329153]
22 16 -7.1741033 (1.3165295, 2.5614555)
[353.31652951 258.56145549]
22 6

In [ ]:
resimg = visual("/home/cp/Desktop/test1.png", kps)

In [ ]:
resimg

In [ ]:
def extract_keypoints(resdata):
    if len(resdata.shape)==4:
        resdata = resdata[0]
    kps = []
    for i in range(14):
        tmpdata = resdata[:,:,i]
        pos = np.unravel_index(np.argmax(tmpdata),tmpdata.shape)
        kps.append(pos)
    return kps

In [ ]:
def visual(imgpath, kps):
    timg = Image.open(imgpath)
    w,h = timg.size
    draw = ImageDraw.Draw(timg)
    for row, col in kps:
        row = int(row/17.0*w)
        col = int(col/23.0*h)
        draw.arc((col-2,row-2, col+2, row+2), 0,360, fill='red', width=1)
    return timg

In [7]:
# refer this code:  https://github.com/rwightman/posenet-python/blob/master/posenet/decode_multi.py

In [8]:
# blog: https://towardsdatascience.com/optimizing-pose-estimation-on-the-coral-edge-tpu-d331c63cfed

In [9]:
# another proach is to use js in android